In [27]:
# 카카오 로컬 api 도로명주소로부터 위도 경도 정보 구하기

In [39]:
import requests
from urllib.parse import quote
import pandas as pd 

In [29]:
with open ('kakaoapikey.txt', mode='r') as key_file:
    kakao_key = key_file.read()

In [30]:
kakao_key

'd837e2a5320287491d0147204e16cf9a'

In [31]:
len(kakao_key)

32

In [32]:
addr = '서울특별시 중구 청파로 463 (중림동)'
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%EC%B2%AD%ED%8C%8C%EB%A1%9C%20463%20%28%EC%A4%91%EB%A6%BC%EB%8F%99%29'

In [33]:
#"Authorization: KakaoAK {REST_API_KEY}"
result = requests.get(url, headers={"Authorization": f"KakaoAK {kakao_key}"}).json()
result

{'documents': [{'address': {'address_name': '서울 중구 중림동 441',
    'b_code': '1114017100',
    'h_code': '1114068000',
    'main_address_no': '441',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '중림동',
    'region_3depth_name': '중림동',
    'sub_address_no': '',
    'x': '126.967109652664',
    'y': '37.5598688323642'},
   'address_name': '서울 중구 청파로 463',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 청파로 463',
    'building_name': '한국경제신문사',
    'main_building_no': '463',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '중림동',
    'road_name': '청파로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.967109652664',
    'y': '37.5598688323642',
    'zone_no': '04505'},
   'x': '126.967109652664',
   'y': '37.5598688323642'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [43]:
lng = result['documents'][0]['x']
lat = result['documents'][0]['y']    # 위도 경도
lng, lat

('126.967109652664', '37.5598688323642')

In [35]:
# addr.csv로부터 주소 읽어서 위도 경도 정보 구한뒤 addr2.csv로 저장해라

In [44]:
df = pd.read_csv('addr.csv')
df

,건물명,도로명주소
0,한국경제신문사,서울특별시 중구 청파로 463 (중림동)
1,서울시청,서울특별시 중구 덕수궁길 15 (서소문동)
2,중구청,서울특별시 중구 창경궁로 17 (예관동)
3,종로구청,서울특별시 종로구 종로1길 36 (수송동)
4,서대문구청,서울특별시 서대문구 연희로 247 (연희동)
5,용산구청,서울특별시 용산구 백범로79길 64 (청파동3가)


In [47]:
lng_list, lat_list = [], []
for i in df.index:
    url = f"{local_url}?query={quote(df['도로명주소'][i])}"
    result = requests.get(url, headers={"Authorization": f"KakaoAK {kakao_key}"}).json()
    lng = result['documents'][0]['x']
    lat = result['documents'][0]['y']  
    lng_list.append(lng)
    lat_list.append(lat)

In [48]:
df['위도'] =lat_list
df['경도'] =lng_list
df

,건물명,도로명주소,위도,경도
0,한국경제신문사,서울특별시 중구 청파로 463 (중림동),37.5598688323642,126.967109652664
1,서울시청,서울특별시 중구 덕수궁길 15 (서소문동),37.5645541852439,126.975613605737
2,중구청,서울특별시 중구 창경궁로 17 (예관동),37.5641201543296,126.998009728978
3,종로구청,서울특별시 종로구 종로1길 36 (수송동),37.5735051436739,126.978988255925
4,서대문구청,서울특별시 서대문구 연희로 247 (연희동),37.5791549275777,126.935815915686
5,용산구청,서울특별시 용산구 백범로79길 64 (청파동3가),37.5416021897984,126.96497660839


In [49]:
df.to_csv('addr2.csv', index=False, sep=',', encoding='utf8')